In [4]:
#Import Libraries
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, KMeans, AffinityPropagation
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# Load the dataset
biomass_data = pd.read_csv('site_data.csv')

Analysis can be done by 1) clustering products, or by 2)clustering on location, and then identifying which location clusters products appear in. To complete 1) we work with the transposed dataframe, where products are on the rows. To complete 2), work with the data frame as is.

## 1) clustering on products

In [5]:
#1) Clustering products

# Transpose the DataFrame to have products as rows and locations as columns
data_transposed = biomass_data.set_index(['dataset', 'site_name', 'site_id', 'X', 'Y']).T

# Remove non-numeric rows
data_transposed_numeric = data_transposed.select_dtypes(include=[np.number])

# Replace 'NA' with NaN
data_transposed_numeric = data_transposed_numeric.replace('NA', np.nan)

# Drop rows with all NaN values
data_transposed_numeric = data_transposed_numeric.dropna(axis=1, how='all')

# Impute missing values with column mean
imputer = SimpleImputer(strategy='mean')
# Impute missing values with ROW mean (must use transposed df and axis=1 for rows)
data_imputed = data_transposed_numeric.T.fillna(data_transposed_numeric.mean(axis=1)).T

In [ ]:
# Perform hierarchical clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# You may need to adjust the number of clusters based on your data and analysis goals
# ‘ward’ minimizes the variance of the clusters being merged.
n_clusters = 3
cluster = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
clusters = cluster.fit_predict(data_scaled)

In [ ]:
# Add cluster labels to the transposed DataFrame, for 1)
data_imputed.loc[:,'cluster'] = clusters

# Transpose the data back to the original format
biomass_data_clustered = data_imputed.reset_index()
biomass_data_clustered

In [ ]:
# Save the clustering results to a CSV file
biomass_data_clustered.to_csv('clustering_product_hierarchical.csv', index=False)